# Currency risks for Swiss residents


## CMD: A Short Summary

If one wants to easily replicate the main findings of this paper, they might execute the command lines below (from the root folder). Later, we'll explore each part in further detail and explore the more nuanced analysis allowed by our implementations.

In [ ]:
:: Setup
setup.bat

:: Data retrieval and processing
python src/data/fred_data_retrieval.py
python src/data/oecd_data_retrieval.py
python src/data/fred_oecd_data_concatenation.py

:: Analysis, modeling and reporting
Rscript src/models/CODE_FAMA_CURRENCY.R
python src/analysis/performances.py

:: LaTeX files compiling

## Data retrieval

We use FRED to gather ForEx and VIX monthly data through FRED's API. We additionally gather German and Italian 10y-bond yields.

We obtain short-term interest rates through an html request to the OECD database.

CAUTION: OECD data for Japan (JPY) is available only since April 2002. Thus, it's not recommended to use a start date previous to this one.

### FRED

In [14]:
import pandas as pd
import src.data.fred_data_retrieval as fred

# Define a time horizon
start_date = pd.Timestamp('2002-04-01')
end_date = pd.Timestamp('2023-10-31')

# Enter FRED API key
api_key = '41641234dac871704cd09b2b8dd163ac'

# Call to download function
fred.download(start_date=start_date,
              end_date=end_date,
              api_key=api_key)


### OECD

In [ ]:
import src.data.oecd_data_retrieval as oecd

oecd.download(start_date=start_date,
              end_date=end_date)

### Merge

In [16]:
import src.data.fred_oecd_data_concatenation as merge
merge.merge()

### Load data

Now, we'll open the downloaded monthly data at a pandas Data Frame:

In [ ]:
df = pd.read_csv('data/processed/monthly_data.csv')

# Standardize data handling with pandas Timestamp
df['Date'] = df['Date'].map(lambda x: pd.Timestamp(x))
df.set_index('Date', inplace=True)

# Visualize the requested data
df.head(6)[['USD', 'GBP', 'EUR', 'VIX', 'USDi', 'GBPi', 'EURi']]

## Analysis

### Regression results
We use R to compute the (log) forward rates and returns, and subsequently perform the linear regression in analysis.

The following command line reproduces the regression and publishes the results onto the LaTeX report, accessible at latex/paper.

In [ ]:
!Rscript src/models/CODE_FAMA_CURRENCY.R 

### FX rates in CHF

The following plot reproduces the FX rates for the specified countries list:

In [12]:
import src.analysis.performances as plot

currencies_list = ['USD', 'GBP', 'JPY']
df2 = plot.build_df(currencies_list)
returns = plot.compute_returns(df2, currencies_list=currencies_list)

In [ ]:
plot.plot_performances(returns)

### Excess returns on Carry FX strategy

We also analyze the profitability and risks associated with investing in the short-term rates in each of the other G-10 currencies, from the standpoint of a Swiss investor.

In [ ]:
carry_fx_xs_returns = plot.compute_carry_fx_xs_returns(df2, currencies_list=currencies_list)
plot.plot_performances(carry_fx_xs_returns)

### Performance indicators

For each of the considered strategies, one might compute historical performance measures such as the Sharpe ratio, the average annualized return and the average and maximum drawdowns through the following code. It's also possible to make drawdown plots, histograms of returns and autocorrelation bar plots.

In [ ]:
plot.plot_return_histograms(returns, currencies_list=currencies_list)
plot.plot_return_histograms(carry_fx_xs_returns, currencies_list=currencies_list)

plot.plot_autocorrelations(carry_fx_xs_returns, currencies_list=currencies_list)

### EU risk factor: DE-IT 10y Spread

The following is an indicator to be used in a robustness check of our results; it's a measure of the European risk.

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
df['IT_minus_DE'].plot(xlabel='Time',
                       ylabel='pp',
                       title='IT-DE 10-year bond spread')
plt.savefig('latex/figures/it_de_spread.jpg')

## LaTeX

Now, let's go through the compiling process of the LaTeX file into a pdf: